In [1]:
%run run_model.py --process=3 --rawname=1 --resume=1 --anchoring=1

24-02-08 06:55:39|MOD:run_model   |: Model Specifics:
24-02-08 06:55:39|MOD:run_model   |: Start Process [Load Data]!


--Process Queue : Data + Instance
--Confirm Resume Training!
--Model_name is set to TRA_LSTM_day!
{'verbosity': 2,
 'storage_type': 'mem',
 'device': device(type='cuda', index=0),
 'precision': 'float',
 'batch_size': 10000,
 'model_name': 'TRA_LSTM_day',
 'model_module': 'TRA_LSTM',
 'model_data_type': 'day',
 'model_num': 1,
 'beg_date': 20170103,
 'end_date': 99991231,
 'interval': 120,
 'input_step_day': 5,
 'test_step_day': 1,
 'MODEL_PARAM': {'hidden_dim': [64],
                 'seqlens': [{'day': 40, '15m': 20, 'dms': 40}],
                 'tra_seqlens': [{'hist_loss': 40}],
                 'rnn_layers': [2],
                 'mlp_layers': [1],
                 'dropout': [0.1],
                 'fc_in': [True],
                 'fc_att': [False],
                 'type_rnn': ['lstm'],
                 'rnn_att': [False],
                 'num_output': [1],
                 'kernel_size': [3, 3],
                 'hidden_as_factor': [False],
                 'ordered_param_gr

24-02-08 06:55:43|MOD:run_model   |: Finish Process [Load Data]! Cost 3.2Secs
24-02-08 06:55:43|MOD:run_model   |: Start Process [Copy to Instance]!
24-02-08 06:55:43|MOD:run_model   |: Copy from model to instance finished , Start going forward


Pre-Norming method of [day] : [endpoint_division(True) , history_standardize(True)]
score function of [pearson] calculated and success!


20170103     0.1458  0.1442  0.1438
20170704     0.1285  0.1275  0.1259
20171226     0.1450  0.1483  0.1458
20180627     0.1281  0.1280  0.1259
20181220     0.0968  0.0947  0.0962
20190624     0.0945  0.0961 -0.0072
20191217     0.0889  0.0884  0.0874
20200617     0.0747  0.0755  0.0754
20201214     0.0979  0.0989  0.0980
20210615     0.0462  0.0471  0.0477
20211209     0.0965  0.0943  0.0963
20220613     0.0803  0.0825  0.0199
20221206     0.0569  0.0598  0.0591
20230606     0.0640  0.0675  0.0693
20231201     0.0949  0.0932  0.0935
AllTimeAvg   0.0960  0.0966  0.0847
AllTimeSum   163.55  164.59  144.28
Std          0.0772  0.0757  0.0816
TValue        51.33   52.67   42.83
AnnIR        6.0915  6.2503  5.0828
24-02-08 06:59:16|MOD:run_model   |: Finish Process [Copy to Instance]! Cost 213.0 Secs


In [6]:
from run_model import RunModel
from copy import deepcopy
import torch
import torch.nn as nn

config , net , batch_data , model_data , metrics , multiloss = RunModel.new_random_input('simple_lstm' , 'day')
optimizer = RunModel.new_optimizer(net)



use d:\Coding\learndl\learndl\scripts\util/../../data/torch_pack/day+rtn11+res11.20231222.pt
Pre-Norming method of [day] : [endpoint_division(True) , history_standardize(True)]


In [7]:
net2 = deepcopy(net)

In [8]:
#pipeline 1

sd = deepcopy(net.state_dict())
optimizer = RunModel.new_optimizer(net)
optimizer.zero_grad()
if hasattr(net , 'dynamic_data_assign'): net.dynamic_data_assign(batch_data , model_data)
pred , hidden = net(batch_data['x'])
penalty_kwargs = {'net' : net , 'hidden' : hidden , 'label' : batch_data['y']}
metric = RunModel.calculate_metrics('train' , metrics, label=batch_data['y'], pred=pred,
                            weight=batch_data['w'], multiloss=multiloss,net=net,valid_sample=None,
                            penalty_kwargs=penalty_kwargs)
print(metric)
(metric['loss'] + metric['penalty']).backward()
print(net.get_parameter('fc.weight').grad)
clip_value = config.train_params['trainer']['gradient'].get('clip_value')
if clip_value is not None : nn.utils.clip_grad_value_(net.parameters(), clip_value = clip_value)
optimizer.step()
sd_step = deepcopy(net.state_dict())


{'loss': tensor(0.9836, grad_fn=<ExpBackward0>), 'loss_item': 0.983590841293335, 'score': 0.016545256599783897, 'penalty': 0.0, 'losses': None}
tensor([[ 0.0346, -0.0963, -0.0509,  0.0964,  0.0248, -0.0379, -0.1012,  0.0861,
          0.0339, -0.0355,  0.0143, -0.0078, -0.0076, -0.0438,  0.1430, -0.0164,
          0.0045, -0.0180,  0.0081, -0.0945, -0.0632,  0.0245, -0.0833, -0.1091,
          0.0300, -0.0056,  0.0512, -0.0070, -0.0515,  0.0182, -0.0090,  0.0971,
         -0.0975, -0.0578, -0.0407, -0.0361, -0.0425,  0.0118, -0.0330,  0.0438,
         -0.0207, -0.1619,  0.0648, -0.0720,  0.0446, -0.0255,  0.0405, -0.0399,
         -0.0412, -0.0356,  0.0476,  0.0033, -0.0390,  0.0607,  0.0745,  0.0257,
          0.0352,  0.0615, -0.1354, -0.0018, -0.0071, -0.1329, -0.0302, -0.0660]])


In [11]:
#pipeline 2
sd2 = deepcopy(net2.state_dict())
optimizer2 = RunModel.new_optimizer(net2)
if hasattr(net2 , 'dynamic_data_assign'): net2.dynamic_data_assign(batch_data , model_data)
pred , hidden = net2(batch_data['x'])
penalty_kwargs = {'net' : net2 , 'hidden' : hidden , 'label' : batch_data['y']}
metric = RunModel.calculate_metrics('train' , metrics, label=batch_data['y'], pred=pred,
                            weight=batch_data['w'], multiloss=multiloss,net=net2,valid_sample=None,
                            penalty_kwargs=penalty_kwargs)
optimizer2.zero_grad()
print(metric)
(metric['loss'] + metric['penalty']).backward()
print(net2.get_parameter('fc.weight').grad)
clip_value = config.train_params['trainer']['gradient'].get('clip_value')
if clip_value is not None : nn.utils.clip_grad_value_(net2.parameters(), clip_value = clip_value)
optimizer2.step()
sd2_step = deepcopy(net2.state_dict())


{'loss': tensor(0.9836, grad_fn=<ExpBackward0>), 'loss_item': 0.983590841293335, 'score': 0.016545256599783897, 'penalty': 0.0, 'losses': None}
tensor([[ 0.0346, -0.0963, -0.0509,  0.0964,  0.0248, -0.0379, -0.1012,  0.0861,
          0.0339, -0.0355,  0.0143, -0.0078, -0.0076, -0.0438,  0.1430, -0.0164,
          0.0045, -0.0180,  0.0081, -0.0945, -0.0632,  0.0245, -0.0833, -0.1091,
          0.0300, -0.0056,  0.0512, -0.0070, -0.0515,  0.0182, -0.0090,  0.0971,
         -0.0975, -0.0578, -0.0407, -0.0361, -0.0425,  0.0118, -0.0330,  0.0438,
         -0.0207, -0.1619,  0.0648, -0.0720,  0.0446, -0.0255,  0.0405, -0.0399,
         -0.0412, -0.0356,  0.0476,  0.0033, -0.0390,  0.0607,  0.0745,  0.0257,
          0.0352,  0.0615, -0.1354, -0.0018, -0.0071, -0.1329, -0.0302, -0.0660]])


In [30]:
for (k,v),(k2,v2) in zip(sd_step.items(),sd2_step.items()):
    print(k , k2 , (v == v2).all())

lstm.weight_ih_l0 lstm.weight_ih_l0 tensor(True)
lstm.weight_hh_l0 lstm.weight_hh_l0 tensor(True)
lstm.bias_ih_l0 lstm.bias_ih_l0 tensor(True)
lstm.bias_hh_l0 lstm.bias_hh_l0 tensor(True)
fc.weight fc.weight tensor(True)
fc.bias fc.bias tensor(True)


In [22]:
class Test():
    @staticmethod
    def aa(x):
        print(x)

    @staticmethod
    def bb(x):
        print(x)

    def trytry(self , y = 1):
        Test.aa(y)




a = Test()
a.trytry(12)

12


In [2]:
class MyClass:
    def static_method1(self):
        print("This is static method 1")
        self.static_method2()

    @classmethod
    def static_method2(cls , x : int = 0) -> None:
        print(f"This is static method 2")
        print(x)
        return None

# 调用静态方法1
a = MyClass()
a.static_method1()
MyClass.static_method2(x=1)


This is static method 1
This is static method 2
0
This is static method 2
1


In [4]:
import numpy as np
import random
pool = np.arange(100)
random.shuffle(pool) # type: ignore

In [8]:
(lambda x=0:1)(3)

1

In [25]:
sd_step

OrderedDict([('lstm.weight_ih_l0',
              tensor([[ 0.0910,  0.0800,  0.0628,  0.0726, -0.1145, -0.1069],
                      [ 0.0649, -0.1171, -0.1229,  0.0789, -0.0136, -0.1013],
                      [-0.0654,  0.0215, -0.0326, -0.0674, -0.0718, -0.0181],
                      ...,
                      [-0.0225, -0.0093, -0.0572,  0.1153,  0.0017,  0.0702],
                      [ 0.1080,  0.1080,  0.1071,  0.0696, -0.0466,  0.0816],
                      [ 0.0576, -0.0698,  0.0820,  0.1018, -0.0175,  0.0566]])),
             ('lstm.weight_hh_l0',
              tensor([[-0.0280, -0.0936,  0.0853,  ..., -0.0813, -0.1271, -0.1084],
                      [-0.0192, -0.0755,  0.0637,  ...,  0.0753,  0.0598,  0.0185],
                      [ 0.0307,  0.0137,  0.0812,  ..., -0.0837, -0.0918, -0.0508],
                      ...,
                      [-0.0289,  0.0206, -0.0827,  ...,  0.1069,  0.0243, -0.1109],
                      [-0.0859,  0.0232, -0.0917,  ..., -0.0174, -0.0

In [ ]:
for (k,v),(k2,v2) in zip(sd.items(),sd2.items()):
    print(k , k2 , (v == v2).all())

In [71]:
optimizer.zero_grad()